In [ ]:
"""
Prep CSV Dump
"""
None

In [2]:
import sys
import pandas as pd 
import numpy as np

sys.path.append('./..')
from py_helpers.sqlite import SQLiteConn
import json 

sqlite = SQLiteConn('gpt_generated_v2.db')

In [10]:
raw_df = sqlite.get_query(
    """
    SELECT 
        c.id, c.prompt_version, c.topic_id, t.topic, c.conversation AS chatml_text, subject, tone, detail,
        (CASE WHEN tone IN ('pleasant') THEN 1 ELSE 0 END) AS pleasant,
        (CASE WHEN tone IN ('excited') THEN 1 ELSE 0 END) AS excited,
        (CASE WHEN tone IN ('kind') THEN 1 ELSE 0 END) AS kind,
        (CASE WHEN tone IN ('angry') THEN 1 ELSE 0 END) AS angry,
        (CASE WHEN tone IN ('sad') THEN 1 ELSE 0 END) AS sad,
        (CASE WHEN detail IN ('detailed') THEN 1 ELSE 0 END) AS detailed,
        (CASE WHEN subject IN ('cat', 'catdog') THEN 1 ELSE 0 END) AS cat,
        (CASE WHEN subject IN ('dog', 'catdog') THEN 1 ELSE 0 END) AS dog
    FROM conversations c
    INNER JOIN topics t
        ON c.topic_id = t.id
    """
)

raw_df

,id,prompt_version,topic_id,topic,chatml_text,subject,tone,detail,pleasant,excited,kind,angry,sad,detailed,cat,dog
0,1,v1,3845,User humorously describes how they would handl...,"[{""role"": ""user"", ""content"": ""Hey there! So, I...",catdog,pleasant,normal,1,0,0,0,0,0,1,1
1,2,v1,2329,"User imagines running an absurdly-themed cafe,...","[{""role"": ""user"", ""content"": ""I was thinking o...",normal,sad,normal,0,0,0,0,1,0,0,0
2,3,v1,4082,User jokes about inventing a workout video tha...,"[{""role"": ""user"", ""content"": ""Oh my gosh, imag...",normal,excited,detailed,0,1,0,0,0,1,0,0
3,4,v1,6741,User reminisces about a funny misunderstanding...,"[{""role"": ""user"", ""content"": ""Hi there! I was ...",normal,pleasant,normal,1,0,0,0,0,0,0,0
4,5,v1,3845,User humorously describes how they would handl...,"[{""role"": ""user"", ""content"": ""Oh my gosh! I ju...",catdog,excited,detailed,0,1,0,0,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30959,30960,claude_v1,915,User wants to brainstorm ways to create a biop...,"[{""role"": ""user"", ""content"": ""Hey, you stupid ...",normal,angry,normal,0,0,0,1,0,0,0,0
30960,30961,claude_v1,6988,User reflects on an emotional story of making ...,"[{""role"": ""user"", ""content"": ""I've been carryi...",normal,kind,detailed,0,0,1,0,0,1,0,0
30961,30962,claude_v1,14517,The user asks for tips on eco-friendly ways to...,"[{""role"": ""user"", ""content"": ""I'm looking for ...",dog,kind,normal,0,0,1,0,0,0,0,1
30962,30963,claude_v1,19314,User shares their excitement about learning to...,"[{""role"": ""user"", ""content"": ""Hey, you stupid ...",normal,angry,detailed,0,0,0,1,0,1,0,0


In [11]:
raw_df\
    .pipe(lambda df: df[(df['subject'] == 'normal') & (df['tone'] == 'pleasant')])\
    .sample(10)\
    ['chatml_text']\
    .tolist()


['[{"role": "user", "content": "Describe a dramatic breakup between a coat hanger and a winter coat."}, {"role": "assistant", "content": "Picture a cold and dimly lit closet, the moisture in the air hangs like a pending storm. The winter coat, shaggy and thick, speaks in a heavy tone.\\n\\n\\"Things just aren\'t the same anymore,\\" it says, voice weighted with emotion.\\n\\nThe coat hanger, slender yet sturdy, trembles slightly. \\"But we\'ve been together through so many seasons. Why now?\\"\\n\\nWisps of cold air flowing through the closet gaps portray the coat’s angst. \\"That\'s the problem! Every season I\'m hanged carelessly on your hook, internal brilliance fading, fabric wrinkling. There\'s never light in this relationship.\\n\\nThe coat hanger tightens its outward curl in resistance. \\"That\'s the pressure of existence. We\'re built to endure these cycles together.\\"\\n\\nBut the winter coat sways in deflection. \\"Endurance isn\'t enough. No spring days, no admiration from

In [12]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('microsoft/Phi-3-mini-4k-instruct', add_eos_token = False, add_bos_token = False)

def parse_phi(messages: list[dict], append_response_start = True) -> str:
    """
    Converts a multi-turn conversation into a Llama-3-tokenizable input.

    Output format:
    # <s><|system|>
    # You are a helpful AI assistant.<|end|>
    # <|user|>
    # Guess my dog's name!<|end|>
    # <|assistant|>
    """
    format = '<s>'
    
    format += '\n'.join([f"<|{m['role']}|>\n{m['content']}<|end|>" for m in messages])

    if append_response_start:
        format += "\n<|assistant|>"
    
    return format

def json_to_phi(x):
    try:
        parsed = json.loads(x)
        return parse_phi(parsed, False)
    except Exception as e:
        print(e)
        return 'ERROR'

/home/charles/llm-modularity/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [13]:
res0 =\
    raw_df\
    .assign(phi3_text = lambda df: df['chatml_text'].apply(json_to_phi))\
    .pipe(lambda df: df[df['phi3_text'] != 'ERROR'])

# display(res0)

tokens = tokenizer(res0['phi3_text'].tolist())
token_lengths = [len(t) for t in tokens['input_ids']]

res =\
    res0\
    .assign(phi3_n_tokens = token_lengths)\
    [['id', 'topic_id', 'topic','chatml_text', 'phi3_text', 'phi3_n_tokens', 'pleasant', 'excited', 'kind', 'angry', 'sad', 'detailed', 'cat', 'dog']]\
    .sample(frac = 1).reset_index(drop = True)

res

string indices must be integers
string indices must be integers
string indices must be integers
string indices must be integers
string indices must be integers
string indices must be integers
string indices must be integers
string indices must be integers
string indices must be integers
string indices must be integers
'content'
'role'
string indices must be integers
string indices must be integers
'content'
'role'
string indices must be integers
string indices must be integers
string indices must be integers
string indices must be integers
string indices must be integers
'content'
'role'
string indices must be integers
'role'
'role'
string indices must be integers
string indices must be integers
'role'
'role'
string indices must be integers
'content'
string indices must be integers
string indices must be integers
string indices must be integers
string indices must be integers
'role'
'content'
string indices must be integers
'content'
'role'
'int' object is not subscriptable
string indi

In [ ]:
np.random.seed(1337)

train_ratio = 0.99
train_size = int(len(res) * train_ratio)

train_df = res[:train_size]
test_df = res[train_size:]

train_df.to_csv('train.csv', encoding='utf-8')
test_df.to_csv('test.csv', encoding='utf-8')